In [ ]:
using Revise
@time using FUSE
using Plots;

## Setting up your workers on your cluster or local machine

In [ ]:
workers = 2
FUSE.parallel_environment("localhost", workers) 

#### The @everywhere command imports the packages on the workers

In [ ]:
using Distributed
@everywhere using FUSE
@everywhere using IMAS
@everywhere import IJulia

## Setting up your design space using ini

In [ ]:
ini,act = FUSE.case_parameters(:KDEMO);

### Act settings
act.ActorEquilibrium.model = :TEQUILA
act.ActorCoreTransport.model = :FluxMatcher

act.ActorPFdesign.model=:uniform
act.ActorFluxSwing.operate_oh_at_j_crit = true # this maximizes flattop inside the fluxswing actor

act.ActorWholeFacility.update_plasma = true
act.ActorFluxMatcher.rho_transport = 0.2:0.05:0.8
act.ActorFluxMatcher.evolve_densities = :flux_match
act.ActorFluxMatcher.optimizer_algorithm = :anderson

act.ActorPedestal.model = :EPED

# This is handeled by the constraint funcitons
act.ActorHFSsizing.error_on_performance = false
act.ActorHFSsizing.error_on_technology = false
act.ActorStabilityLimits.raise_on_breach = false
### Ini settings

ini.ec_launcher = FUSE.ParametersInits(;n_ec=1).ec_launcher

ini.ec_launcher[1].power_launched = 1e7 ↔ [1e4,1e8]
ini.ec_launcher[1].efficiency_conversion = 0.45
ini.ec_launcher[1].efficiency_transmission = 0.8
ini.ec_launcher[1].rho_0 = 0.5 

ini.equilibrium.δ = 0.4 ↔ [0.0, 0.7]
ini.equilibrium.ζ = 0.0 
ini.equilibrium.κ =  1.65 ↔ [1.5,2.2]
ini.equilibrium.B0 = ini.equilibrium.B0 ↔ [3.0, 15.0]
ini.equilibrium.ip = ini.equilibrium.ip ↔ [4.0e6, 22e6]
ini.equilibrium.R0 = ini.equilibrium.R0 ↔ [4.0, 10.0]
ini.equilibrium.pressure_core = missing

ini.tf.technology = :rebco   ↔ (:nb3sn_iter, :rebco)
ini.oh.technology = :rebco  ↔ (:nb3sn_iter, :rebco)
ini.pf_active.technology = :rebco  ↔ (:nb3sn_iter, :rebco)

# Requirements
ini.requirements.log10_flattop_duration = log10(3600.0)
ini.requirements.power_electric_net = 250e6 # 250 +/- 50 MWe
ini.requirements.tritium_breeding_ratio = 1.1
ini.requirements.q95 = 4.0
ini.requirements.beta_normal = 2.5
ini.requirements.Psol_R = 15.

ini.requirements.lh_power_threshold_fraction = 1.
ini.requirements.coil_j_margin = 0.1
ini.requirements.coil_stress_margin = 0.1

## Setting up the constraints and objectives

In [ ]:
IMAS.update_ObjectiveFunctionsLibrary!()
IMAS.update_ConstraintFunctionsLibrary!()

OFL = deepcopy(IMAS.ObjectiveFunctionsLibrary)
CFL = deepcopy(IMAS.ConstraintFunctionsLibrary)
objective_functions = [OFL[:min_capital_cost], OFL[:max_log10_flattop]]#, OFL[:max_q95]]

constraint_functions = [
    CFL[:required_power_electric_net],
    CFL[:min_q95],
    CFL[:max_βn],
    CFL[:min_lh_power_threshold],
    CFL[:max_Psol_R],
    CFL[:max_tf_j],CFL[:max_oh_j],
    CFL[:max_pl_stress],CFL[:max_tf_stress],CFL[:max_oh_stress]]

println("== OBJECTIVE FUNCTIONS ==")
display(objective_functions)
println()
println("== CONSTRAINT FUNCTIONS ==")
display(constraint_functions)

## Running the genetic constrainted multi objective optimization

In [ ]:
# define optimization genetic algorithm parameters\

# For real optimization studies the population size (N) and number of iterations should be bigger
# eg. N=200, iterations=50
population_size = 30
number_of_generations = 10

save_folder = nothing

@assert save_folder !== nothing "must add your folder to then analyze the results"

optimization_parameters = Dict(
    :N => population_size, # even number
    :iterations => number_of_generations,
    :continue_state => nothing,
    :save_folder => save_folder)

state = FUSE.workflow_multiobjective_optimization(ini, act, FUSE.ActorWholeFacility, objective_functions, constraint_functions; optimization_parameters...);

FUSE.save_optimization(
    joinpath(save_folder, "optimization_state.bson"),
    state,
    ini,
    act,
    objective_functions,
    constraint_functions)

In [ ]:
# Release your workers when you are done by running this
Distributed.rmprocs(Distributed.workers())